In [64]:
import numpy as np
from mygrad import Tensor
from nltk import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from collections import Counter
from mygrad import Tensor
from mygrad.nnet.layers import dense
from mygrad.nnet.activations import softmax, relu
from mygrad.nnet.losses import multiclass_hinge
import time

In [136]:
from mygrad.math import log
def cross_entropy(p_pred, p_true):
    """ Computes the mean cross-entropy.
        
        Parameters
        ----------
        p_pred : mygrad.Tensor, shape:(N, K)
            N predicted distributions, each over K classes.
        
        p_true : mygrad.Tensor, shape:(N, K)
            N 'true' distributions, each over K classes
        
        Returns
        -------
        mygrad.Tensor, shape=()
            The mean cross entropy (scalar)."""
    
    N = p_pred.shape[0]
    global p_logq
    p_logq = (p_true) * log(p_pred)
    return (-1/ N) * p_logq.sum()  

In [49]:
print(p_logq)
n = p_logq.sum()
n

Tensor(
[[-0.         -0.         -0.         -1.60991107 -0.        ]
 [-0.         -0.         -1.60958464 -0.         -0.        ]
 [-0.         -0.         -1.60912031 -0.         -0.        ]
 ..., 
 [-0.         -0.         -0.         -1.61024727 -0.        ]
 [-0.         -0.         -0.         -1.60964328 -0.        ]
 [-0.         -0.         -0.         -0.         -1.60893789]]
)


Tensor(nan)

In [63]:
np.where(np.isnan(p_logq.data) == True)

(array([4508, 4508, 4508, 4508, 4508, 5209, 5209, 5209, 5209, 5209, 5589,
        5589, 5589, 5589, 5589], dtype=int64),
 array([0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4], dtype=int64))

In [ ]:
import pandas as pd
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('reviews_Movies_and_TV_5.json.gz')
# 1697533 samples

In [24]:
reviews = np.array(df["reviewText"])
ratings = np.array(df["overall"])
y_train = reviews[:900000]
y_val = reviews[900000:1000000]
y_test = reviews[1000000:]

In [27]:
arr = np.zeros(len(reviews), dtype=Counter)
i = 0
for r in reviews:
    tokens = word_tokenize(r)
    count = Counter(tokens)
    arr[i] = count
    i += 1
arr

KeyboardInterrupt: 

In [34]:
#624356
reviews = arr[:50000]
ratings = np.array(df["overall"])[:50000]
y_train = ratings[:20000]
y_val = ratings[20000:25000]
y_test = ratings[25000:]

In [4]:
# SOLUTION
def to_bag(counters, k=None, stopwords=[]):
    instances = Counter()
    for counter in counters:
        instances += counter
    for word in stopwords:
        del instances[word]
    for p in punctuation:
        del instances[p]
    if k is not None and k < len(instances):
        return list(list(zip(*sorted(instances.most_common(k))))[0])
    return sorted(instances)

# to_bag(reviews)

In [36]:
bag = to_bag(reviews)

In [14]:
y_train = ratings[:7000]
y_val = ratings[7000:9000]
y_test = ratings[9000:]

In [48]:
bag_2 = to_bag(reviews, k=10000, stopwords=stopwords.words())

In [49]:
bag_2

["''",
 "'90s",
 "'S",
 "'The",
 "'d",
 "'em",
 "'ll",
 "'m",
 "'re",
 "'s",
 "'the",
 "'ve",
 '***',
 '*****',
 '*The',
 '--',
 '..',
 '...',
 '.A',
 '.And',
 '.As',
 '.I',
 '.If',
 '.In',
 '.It',
 '.The',
 '.This',
 '.and',
 '.the',
 '0',
 '007',
 '1',
 '1.85:1',
 '1/2',
 '10',
 '10/10',
 '100',
 '1000',
 '1080p',
 '11',
 '12',
 '12-year',
 '12-year-old',
 '13',
 '13th',
 '14',
 '15',
 '150',
 '16',
 '165',
 '16:9',
 '17',
 '1776',
 '17th',
 '18',
 '18th',
 '19',
 '1900',
 '1903',
 '1920s',
 '1930',
 '1930s',
 '1936',
 '1939',
 '1940',
 '1941',
 '1943',
 '1944',
 '1945',
 '1946',
 '1950',
 '1950s',
 '1951',
 '1954',
 '1958',
 '1959',
 '1960',
 '1960s',
 '1961',
 '1962',
 '1964',
 '1965',
 '1966',
 '1967',
 '1968',
 '1969',
 '1970',
 '1970s',
 '1971',
 '1972',
 '1973',
 '1974',
 '1975',
 '1976',
 '1977',
 '1978',
 '1979',
 '1980',
 '1980s',
 '1981',
 '1982',
 '1983',
 '1984',
 '1985',
 '1986',
 '1987',
 '1988',
 '1989',
 '1990',
 '1990s',
 '1991',
 '1992',
 '1993',
 '1994',
 '1995',
 

In [105]:
def to_tf(counts, bow):
    desc = []
    for word in bow:
        desc.append(counts[word])
    desc = np.array(desc)
    desc[np.where(desc is None)] = 0
    if sum(desc) != 0:
        des = desc / sum(desc)
    else:
        des = desc
    return des

In [110]:
tfs = [to_tf(r, bag_2) for r in reviews]

In [6]:
# SOLUTION
def idf(bow, counts):
    idfs = []
    total = len(counts)
    for word in bow:
        idfs.append(total / np.count_nonzero([g[word] for g in counts]))
    return np.log10(np.array(idfs))

In [87]:
def tfidf(tf, idf):
    return tf * idf

In [85]:
idfs = idf(bag_2, reviews)

In [8]:
def cos_dist(a, b):
    abd = a.dot(b)
    mag_a = np.sqrt(np.sum(a ** 2))
    mag_b = np.sqrt(np.sum(b ** 2))
    return (abd / (mag_a * mag_b))

In [111]:
tf_idf = tfidf(tfs, idfs)

In [68]:
y = 

10000

In [112]:
inputs = tf_idf
x_train = inputs[:7000]
x_val = inputs[7000:9000]
x_test = inputs[9000:]

In [89]:
np.where(np.isnan(tfs))

(array([4508, 4508, 4508, ..., 9208, 9208, 9208], dtype=int64),
 array([   0,    1,    2, ..., 9997, 9998, 9999], dtype=int64))

In [10]:
def compute_accuracy(model_out, labels):
    """ Computes the mean accuracy, given predictions and true-labels.
        
        Parameters
        ----------
        model_out : numpy.ndarray, shape=(N, K)
            The predicted class-scores/probabilities
        labels : numpy.ndarray, shape=(N,)
            The labels for the data.
        
        Returns
        -------
        float
            The mean classification accuracy of the N samples."""
    return np.mean(np.argmax(model_out, axis=1) == labels)

In [11]:
def sgd(param, rate):
    """ Performs a gradient-descent update on the parameter.
    
        Parameters
        ----------
        param : mygrad.Tensor
            The parameter to be updated.
        
        rate : float
            The step size used in the update"""
    param.data -= rate*param.grad
    return None

In [149]:
def he_normal(shape):
    """ Given the desired shape of your array, draws random
        values from a scaled-Gaussian distribution.
        
        Returns
        -------
        numpy.ndarray"""
    N = shape[0]
    scale = 1 / np.sqrt(2*N)
    return np.random.randn(*shape)#*scale

In [109]:
import pickle
with open("save_data.txt", "rb") as f:
    reviews, ratings, inputs, bag_2 = pickle.load(f)

In [113]:
with open("save_data.txt", "wb") as f:
    pickle.dump([reviews, ratings, inputs, bag_2], f)

In [144]:
inputs

array([[ 0.02067323,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ..., 
       [ 0.00800254,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [153]:
def train(x, y, xval, yval, step=1, iters=400):
    w1 = Tensor(he_normal((x.shape[1], 2000)))
    print(w1[0, 0])
    b1 = Tensor(np.zeros(2000, dtype=w1.dtype))
    w2 = Tensor(he_normal((2000, 300)))
    b2 = Tensor(np.zeros(300, dtype=w2.dtype))
    w3 = Tensor(he_normal((300, 5)))
    b3 = Tensor(np.zeros(5, dtype=w3.dtype))
    params = [b1, w1, b2, w2, b3, w3]

    y_cross = np.zeros((x.shape[0], 5))
    y_cross[np.arange(x.shape[0]), y] = 1
    
    accuracies = []
    losses = []
    t0 = time.time()
    for i in range(iters):
        if i % 10 == 0:
            print(i, time.time() - t0)
        o1 = relu(dense(x, w1) + b1)
        o2 = relu(dense(o1, w2) + b2)
        y_pred = softmax(dense(o2, w3) + b3)
        print(np.where(y_pred < 0))
        
        loss = cross_entropy(y_pred, y_cross)
        losses.append(loss.data.item())
        loss.backward()

        accuracies.append(compute_accuracy(y_pred.data, y))

        for param in params:
            sgd(param, step)
        print(w1[0, 0])

        loss.null_gradients()
    
    o1 = relu(dense(x, w1) + b1)
    o2 = relu(dense(o1, w2) + b2)
    train_pred = softmax(dense(o2, w3) + b3)
    train_acc = compute_accuracy(train_pred, y)
    
    o1 = relu(dense(xval, w1) + b1)
    o2 = relu(dense(o1, w2) + b2)
    val_pred = softmax(dense(o2, w3) + b3)
    val_acc = compute_accuracy(val_pred, yval)
    print("Training accuracy:", train_acc, "; Validation accuracy:", val_acc)
    return train_acc, val_acc, accuracies, losses, params

In [154]:
ta, va, acc, los, pms = train(x_train, y_train, x_val, y_val, iters=20)

Tensor(0.07294358180734294)
0 0.0
(array([], dtype=int64), array([], dtype=int64))


ValueError: Invalid log-domain value

In [134]:
acc

[0.38514285714285712,
 0.60999999999999999,
 0.60999999999999999,
 0.60999999999999999,
 0.60999999999999999,
 0.60999999999999999,
 0.60999999999999999,
 0.60999999999999999,
 0.60999999999999999,
 0.60999999999999999,
 0.60999999999999999,
 0.60999999999999999,
 0.60999999999999999,
 0.60999999999999999,
 0.60999999999999999,
 0.60999999999999999,
 0.60999999999999999,
 0.60999999999999999,
 0.60999999999999999,
 0.60999999999999999]